In [2]:
import numpy as np
import networkx as nx
import pandas as pd
import time, os
from collections import deque
from numba import njit
from joblib import Parallel, delayed

# =========================================================
# GRAPH BUILDERS & PRECOMPUTE
# =========================================================
def make_graph(topo, N, z, seed):
    if topo == "BA":
        m = int(z / 2)
        return nx.barabasi_albert_graph(N, m, seed=seed)
    elif topo == "ER":
        return nx.erdos_renyi_graph(N, z / (N - 1), seed=seed)
    else:
        raise ValueError("topo must be 'BA' or 'ER'")

def make_influence_graph(G_g, corr, N, seed_perm=777):
    if corr == "max": return G_g
    rng = np.random.default_rng(seed_perm)
    perm = np.arange(N, dtype=np.int32)
    rng.shuffle(perm)
    mapping = {int(i): int(perm[i]) for i in range(N)}
    return nx.relabel_nodes(G_g, mapping)

def graph_to_csr(G):
    G = nx.convert_node_labels_to_integers(G)
    N = G.number_of_nodes()
    adj_f = []
    adj_o = [0]
    for i in range(N):
        neigh = list(G.neighbors(i))
        adj_f.extend(neigh)
        adj_o.append(len(adj_f))
    return np.array(adj_f, dtype=np.int32), np.array(adj_o, dtype=np.int32), (np.array(adj_o[1:]) - np.array(adj_o[:-1])).astype(np.int32)

def build_influence_layers_bfs(G_i, N, Lmax):
    adj = [list(G_i.neighbors(i)) for i in range(N)]
    inf_indices = np.zeros((Lmax, N, N), dtype=np.int32)
    inf_counts  = np.zeros((Lmax, N), dtype=np.int32)
    for src in range(N):
        dist = np.full(N, -1, dtype=np.int32)
        dist[src] = 0
        q = deque([src])
        while q:
            u = q.popleft()
            if dist[u] == Lmax: continue
            for v in adj[u]:
                if dist[v] == -1:
                    dist[v] = dist[u] + 1
                    q.append(v)
        for j in range(N):
            d = dist[j]
            if 0 < d <= Lmax:
                l = d - 1
                c = inf_counts[l, src]
                inf_indices[l, src, c] = j
                inf_counts[l, src] += 1
    return inf_indices, inf_counts

# =========================================================
# NUMBA CORE - COST AS PERCENTAGE
# =========================================================
@njit(cache=True, fastmath=True)
def run_simulation_percentage_cost(
    adj_f, adj_o, deg_f,
    inf_indices, inf_counts,
    alpha_weights,
    theta, L, b,
    steps, update_rule, K, vigilance_mode,
    cost_percentage, # 0.1 = 10% coût, -0.1 = 10% récompense
    seed):
    
    np.random.seed(seed)
    N = deg_f.shape[0]

    # Stratégies: 1=Coop, 0=Def
    strat = np.random.randint(0, 2, N).astype(np.float64)
    vig = np.zeros(N, dtype=np.float64)
    for i in range(N):
        if strat[i] == 1.0 and np.random.random() < 0.5:
            vig[i] = 1.0

    influences = np.empty(N, dtype=np.float64)
    T_eff = np.empty(N, dtype=np.float64)
    payoffs = np.empty(N, dtype=np.float64)
    new_strat = np.empty(N, dtype=np.float64)

    history_window = 300
    burn_in = 1200
    tol_std = 8e-4
    history_c = np.zeros(history_window, dtype=np.float64)
    filled = 0

    for t in range(steps):
        # A) Influence
        for i in range(N):
            I_i = 0.0
            for l_idx in range(L):
                cnt = inf_counts[l_idx, i]
                if cnt > 0:
                    v_sum = 0.0
                    for k in range(cnt): v_sum += vig[inf_indices[l_idx, i, k]]
                    I_i += alpha_weights[l_idx] * (v_sum / cnt)
            influences[i] = I_i

        # B) Vigilance
        if vigilance_mode == 0: # amnesic
            for i in range(N):
                vig[i] = 1.0 if (strat[i] == 1.0 and influences[i] >= theta) else 0.0
        else: # memory
            for i in range(N):
                if strat[i] == 0.0: vig[i] = 0.0
                elif vig[i] == 0.0: vig[i] = 1.0 if influences[i] >= theta else 0.0

        # C) Payoffs de base
        for i in range(N):
            T_eff[i] = 1.0 + (b - 1.0) * (1.0 - influences[i])
            payoffs[i] = 0.0

        for i in range(N):
            start, end = adj_o[i], adj_o[i + 1]
            if strat[i] == 1.0:
                for idx in range(start, end):
                    if strat[adj_f[idx]] == 1.0: payoffs[i] += 1.0
            else:
                Ti = T_eff[i]
                for idx in range(start, end):
                    if strat[adj_f[idx]] == 1.0: payoffs[i] += Ti

        # --- CORRECTION & POURCENTAGE ---
        # On applique le coût/récompense proportionnel au gain
        for i in range(N):
            if vig[i] == 1.0:
                payoffs[i] *= (1.0 - cost_percentage)

        # D) Update Rule
        for i in range(N):
            new_strat[i] = strat[i]
            k_i = adj_o[i + 1] - adj_o[i]
            if k_i == 0: continue
            j = adj_f[adj_o[i] + np.random.randint(0, k_i)]
            
            if update_rule == 0: # Prop
                if payoffs[j] > payoffs[i]:
                    phi = max(k_i, deg_f[j]) * max(T_eff[i], T_eff[j])
                    if phi > 0 and np.random.random() < (payoffs[j]-payoffs[i])/phi:
                        new_strat[i] = strat[j]
            else: # Fermi
                k_noise = max(K, 1e-12)
                p = 1.0 / (1.0 + np.exp((payoffs[i] - payoffs[j]) / k_noise))
                if np.random.random() < p: new_strat[i] = strat[j]

        strat[:] = new_strat[:]
        rho = strat.mean()

        if rho <= 0.0 or rho >= 1.0: return rho, vig.mean(), t, 0
        history_c[t % history_window] = rho
        filled = min(filled + 1, history_window)
        if t > burn_in and t % 50 == 0 and filled == history_window:
            if np.std(history_c) < tol_std: return history_c.mean(), vig.mean(), t, 1

    return strat.mean(), vig.mean(), steps, 2

# =========================================================
# CONFIGURATION & RUN
# =========================================================
N = 500
REPS = 50
steps = 15000 # Un peu plus long pour être "carré"
theta = 0.2
L = 1
K = 0.1
# ICI : On met des pourcentages (0.1 = 10% de taxe, -0.1 = 10% de bonus)
costs_pct = [-0.1, -0.05, 0.0, 0.05, 0.1, 0.2,0.5]
B_RANGE = np.around(np.arange(1.1, 2.1, 0.1), 1)
TOPOS = [("ER", 16),("BA", 4)]

OUTFILE = "Comparaison_final_L1.csv"
IC_SEEDS = np.array([12345 + 1000*r for r in range(REPS)], dtype=np.int64)

for topo, z in TOPOS:
    Gg = make_graph(topo, N, z, seed=42)
    af, ao, deg = graph_to_csr(Gg)
    inf_idx, inf_cnt = build_influence_layers_bfs(Gg, N, L)
    alphas = np.array([0.5**l for l in range(L)]); alphas /= alphas.sum()

    for cp in costs_pct:
        for b in B_RANGE:
            res = Parallel(n_jobs=-1)(delayed(run_simulation_percentage_cost)(
                af, ao, deg, inf_idx, inf_cnt, alphas, theta, L, b, steps, 1, K, 1, cp, IC_SEEDS[r]
            ) for r in range(REPS))
            
            rhos, vigs, tsts, reas = zip(*res)
            pd.DataFrame([{
                "Topo": topo, "Z": z, "b": b, "Cost_Pct": cp,
                "C_mean": np.mean(rhos), "C_std": np.std(rhos),
                "V_mean": np.mean(vigs), "V_std": np.std(vigs),
                "Tstop_mean": np.mean(tsts), "Stop_max_frac": np.mean(np.array(reas) == 2)
            }]).to_csv(OUTFILE, mode="a", header=not os.path.exists(OUTFILE), index=False)
            print(f"[{topo} z={z}] Cost {cp*100}% b={b} done.")


KeyboardInterrupt: 